In [2]:
!pip3 install torch torchvision 

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 13.0 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.15.0 requires datasets>=2.21.0, which is not installed.


In [1]:
# 1. 导入所需要的库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms



In [2]:
# 2. 下载CIFAR-10数据集
# 设置图像预处理: 图像增强 + 转换为张量 + 标准化
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 下载训练集和测试集
print("[LOG] Now loading CIFAR-10 dataset for Training...")
trainset = torchvision.datasets.CIFAR10(root='CIFAR10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
print("[LOG] Now loading CIFAR-10 dataset for Testing...")
testset = torchvision.datasets.CIFAR10(root='CIFAR10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)
print("[LOG] Loading CIFAR-10 dataset finished.")

[LOG] Now loading CIFAR-10 dataset for Training...
[LOG] Now loading CIFAR-10 dataset for Testing...
[LOG] Loading CIFAR-10 dataset finished.


In [3]:
# 3. 使用ResNet-18作为预训练网络
# 下载预训练的ResNet-18模型
print("[LOG] Now loading model RestNet-18...")
resnet18 = torchvision.models.resnet18(pretrained=True)
print("[LOG] Loading model ResNet-18 finished.")
# 由于CIFAR-10有10个类，我们需要调整ResNet的最后一个全连接层
num_classes = 10
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

[LOG] Now loading model RestNet-18...


c:\Users\KEKE\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KEKE\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[LOG] Loading model ResNet-18 finished.


In [4]:
import torch.mps
import torch.xpu


device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda:0'
if(torch.xpu.is_available()):
    device = 'xpu'
if(torch.mps.is_available()):
    device = 'mps'
print("[LOG] Using device: ", device)

[LOG] Using device:  xpu


In [5]:
# 4. 微调预训练的CNN网络
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)



In [6]:
# 迁移到GPU上（如果有的话）
resnet18.to(device)
# 训练网络
print("[LOG] Start training...")
for epoch in range(10):  # 就演示训练10个epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入数据
        inputs, labels = data[0].to(device), data[1].to(device)
        # 清零参数梯度
        optimizer.zero_grad()
        # 前向 + 反向 + 优化
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:  # 每200批次打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('[LOG] Training finished')

[LOG] Start training...
[1,   200] loss: 1.575
[1,   400] loss: 1.160
[1,   600] loss: 1.021
[2,   200] loss: 0.880
[2,   400] loss: 0.843
[2,   600] loss: 0.815
[3,   200] loss: 0.738
[3,   400] loss: 0.715
[3,   600] loss: 0.728
[4,   200] loss: 0.642
[4,   400] loss: 0.657
[4,   600] loss: 0.641
[5,   200] loss: 0.582
[5,   400] loss: 0.607
[5,   600] loss: 0.605
[6,   200] loss: 0.570
[6,   400] loss: 0.565
[6,   600] loss: 0.569
[7,   200] loss: 0.526
[7,   400] loss: 0.528
[7,   600] loss: 0.534
[8,   200] loss: 0.493
[8,   400] loss: 0.502
[8,   600] loss: 0.498
[9,   200] loss: 0.480
[9,   400] loss: 0.481
[9,   600] loss: 0.471
[10,   200] loss: 0.452
[10,   400] loss: 0.460
[10,   600] loss: 0.465
[LOG] Training finished


In [11]:
# 5. 测试网络性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('[LOG] Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

RuntimeError: Native API failed. Native API returns: 2147483646 (UR_RESULT_ERROR_UNKNOWN)